In [ ]:
# dependencies
# possibly more than needed
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import chardet
import gmaps

# Import API key
from api_keys import g_key

In [ ]:
# Make list of column/headers for Great Big Dataframe (GBDf) with empty cells

# FIPS
# (five-digit code: the first two represent state, the last three represent county in that state)

# County
# (name, e.g. Jackson County in many states, but also Orleans Parish of Louisiana, or the Aleutians East Borough of Alaska)

# State (name)

# Pop
# (2019 county population estimate)

# PopDens
# (Population density; derived from county 2019 population estimate divided by county land mass in another dataframe)

# MskScore
# (Mask-wearing score derived from pct in each county who said never, rarely sometimes, frequently, always wear a mask when going out)

# CaseRate
# (cumulative COVID cases per 100,000 population by county January 21–July 14, 2020)

# DeathRate
# (cumulative COVID deaths per 100,000 population by county January 21–July 14, 2020)

# Income
# (Mean per-capita income by county; this is a bonus-round question)

In [ ]:
# Matt's code starts here

# U.S. Census 2010-2019

In [ ]:
# 1. US Census 2010-2019
censusDataReadMeURL = "https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html"

# This is where the census data .CSV lives locally...:
censusDataFilepath = "Resources/co-est2019-alldata_exp.csv"

print(f"{censusDataFilepath} is {round(os.path.getsize(censusDataFilepath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{censusDataReadMeURL}")

In [ ]:
# Read CSV into censusData DataFrame
censusData = pd.read_csv(censusDataFilepath, encoding="iso-8859-1")
censusData

In [ ]:
# 2. COVID-19 cases
caseDataReadMeURL = "https://github.com/nytimes/covid-19-data/blob/master/README.md"

# This is where the .CSV lives locally...:
caseDataFilepath = "Resources/us-counties.csv"

print(f"The file at {caseDataFilepath} is {round(os.path.getsize(caseDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{caseDataReadMeURL}")

In [ ]:
# Read CSV into caseData DataFrame
caseData = pd.read_csv(caseDataFilepath, encoding = "UTF-8")
caseData

In [ ]:
# Drop rows containing NaN values (caseData's "unknown" counties)
caseData.dropna(axis = 0, how = "any", thresh = None, subset = None, inplace = True)
caseData

In [ ]:
# Convert caseData FIPS values from float to int
caseData.fips = caseData.fips.astype(np.int64)
caseData.dtypes

In [ ]:
# Merge caseData and censusData DataFrames
merged_caseData = pd.merge(censusData, caseData, how = "outer", left_on = "FIPS", right_on = "fips", on = None, sort = False, copy = True, indicator = False, validate = None)
merged_caseData

In [ ]:
# Drop duplicate and/or irrelevant columns
merged_caseData.drop(columns = ["FIPS", "STATE", "COUNTY", "STNAME", "CTYNAME", "CENSUS2010POP"], inplace = True)
merged_caseData.head()

In [ ]:
# Manually add population to row 3142 (New York City aggregate)
merged_caseData["POPESTIMATE2019"][3142] = 8336817
merged_caseData.tail()

In [ ]:
# Rearrange columns
organized_caseData = merged_caseData[["date", "fips", "county", "state", "POPESTIMATE2019", "cases", "deaths"]]
organized_caseData.head()

In [ ]:
# Rename columns
renamed_caseData = merged_caseData.rename(columns = {"date":"Date", "fips":"FIPS", "county":"County", "state":"State",
                                                     "POPESTIMATE2019":"PopEst", "cases":"Cases", "deaths":"Deaths"})
renamed_caseData.head()

In [ ]:
# Rearrange columns
reorganized_caseData = renamed_caseData[["Date", "FIPS", "County", "State", "PopEst", "Cases", "Deaths"]]
reorganized_caseData

In [ ]:
# Due diligence to check DataFrame for rows with missing data
reorganized_caseData.count()

In [ ]:
# Drop rows containing no data and reset index
complete_caseData = reorganized_caseData.dropna(axis = 0, how = "any", thresh = None, subset = None, inplace = False)
complete_caseData = complete_caseData.reset_index()
complete_caseData.tail()

In [ ]:
# Create DataFrame containing (4) boroughs of New York (Manhattan dropped out on execution of previous cell) so the
# population data is not counted twice
droppedNYData = complete_caseData.loc[(complete_caseData["State"] == "New York") & (complete_caseData["Cases"] == 0), :]
droppedNYData

In [ ]:
# Remove droppedNYData DataFrame from complete_caseData DataFrame and reset index
final_caseData_index = droppedNYData.index
final_caseData = complete_caseData.drop(final_caseData_index, inplace = False)
final_caseData = final_caseData.reset_index()
final_caseData.tail()

In [ ]:
# Drop duplicate and/or irrelevant columns
final_caseData.drop(columns = ["level_0", "index"], inplace = True)
final_caseData.head()

In [ ]:
final_caseData.count()

In [ ]:
# Convert final_caseData FIPS values from float to int
final_caseData.FIPS = final_caseData.FIPS.astype(np.int64)
final_caseData.dtypes

# U.S. Landmass Data (by County)

In [ ]:
# 3. Landmass (and thence population density)
landMassDataReadMeURL = "https://hub.arcgis.com/datasets/48f9af87daa241c4b267c5931ad3b226_0/data?orderBy=FIPS"
# This is where the land mass data .CSV lives locally...:
landMassDataFilepath = "Resources/counties-by-land-area.csv"

print(f"{landMassDataFilepath} is {round(os.path.getsize(landMassDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{landMassDataReadMeURL}")

In [ ]:
# Read CSV into landmassData DataFrame
landmassData = pd.read_csv(landMassDataFilepath)
landmassData

In [ ]:
# Reset index
landmassData = landmassData.reset_index()
landmassData.tail()

In [ ]:
# Create DataFrame to sort landmassData by FIPS code
a = landmassData[["FIPS", "FID", "NAME", "STATE_NAME", "STATE_NAME", "POPULATION", "SQMI"]]
a = a.sort_values(by = "FIPS").reset_index().drop(columns = ["index"])
a.tail()

In [ ]:
# Create DataFrame to sort final_caseData by FIPS code
b = final_caseData[["Date", "FIPS", "County", "State", "Cases", "Deaths"]]
b = b.sort_values(by = "FIPS").reset_index().drop(columns = ["index"])
b.tail()

In [ ]:
# Convert final_caseData FIPS values to integer and verify data types
b["FIPS"] = b["FIPS"].astype(int)
b.dtypes

In [ ]:
# Verify landmassData data types
a.dtypes

In [ ]:
# Verify last row's index number
b.tail()

In [ ]:
# Calculate difference in rows between final_caseData and landmassData DataFrames to identify Puerto Rico and
# other non-U.S. "counties" we do not have case data for
len(a) - len(b)

In [ ]:
# Create DataFrame to hold duplicate county's case data
c = b[b["FIPS"] == 2016]
c

In [ ]:
# Create DataFrame to hold duplicate county's census data
d = a[a["FIPS"] == 2016]
d

In [ ]:
# Merge DataFrames to create one entry for duplicate county
final_merged_caseData = b.merge(a, how = "left", on = "FIPS")
final_merged_caseData.isnull().sum()

In [ ]:
# Verify merge was successful
check = final_merged_caseData[final_merged_caseData["FIPS"] == 2016]
check

In [ ]:
# View final_merged_caseData DataFrame to verify we have 3132 rows (including New York City aggregate)
final_merged_caseData

In [ ]:
# Drop duplicate and/or irrelevant columns
final_merged_caseData.drop(columns = ["FID", "NAME", "STATE_NAME", "STATE_NAME"], inplace = True)
final_merged_caseData.head()

In [ ]:
# Manually add population and landmass data to row 3132 (New York City aggregate) and verify
final_merged_caseData["POPULATION"][3131] = 8336817
final_merged_caseData["SQMI"][3131] = 302.06
final_merged_caseData.tail()

In [ ]:
# Rename column
final_merged_caseData = final_merged_caseData.rename(columns = {"POPULATION":"PopEst"})
final_merged_caseData

In [ ]:
# Create per 100,000 people divisor
perHundredK_divisor = final_merged_caseData["PopEst"] / 100000

# Calculate cases per 100,000
casesPerHundredK = final_merged_caseData["Cases"] / perHundredK_divisor

# Calculate deaths per 100,000
deathsPerHundredK = final_merged_caseData["Deaths"] / perHundredK_divisor

# Calculate population density
popDens = final_merged_caseData["PopEst"] / final_merged_caseData["SQMI"]

In [ ]:
# Add new columns to hold case rates and death rates (per 100,000 people), and population density
final_merged_caseData["CaseRate"] = casesPerHundredK
final_merged_caseData["DeathRate"] = deathsPerHundredK
final_merged_caseData["PopDens"] = popDens
final_merged_caseData.head()

In [ ]:
# Reorganize columns
reorganized_final_merged_caseData = final_merged_caseData[["Date", "FIPS", "County", "State", "SQMI", "PopEst", "PopDens",
                                                           "Cases", "CaseRate", "Deaths", "DeathRate"]]
reorganized_final_merged_caseData

In [ ]:
# Sort on DeathRate (or CaseRate) to find best/worst counties (for screenshot image)
# reorganized_final_merged_caseData = reorganized_final_merged_caseData.sort_values("DeathRate", ascending=False)
# reorganized_final_merged_caseData

In [ ]:
# Convert reorganized_final_merged_caseData FIPS and Population values from float to int
reorganized_final_merged_caseData.FIPS = reorganized_final_merged_caseData.FIPS.astype(np.int64)
reorganized_final_merged_caseData.PopEst = reorganized_final_merged_caseData.PopEst.astype(np.int64)
reorganized_final_merged_caseData.dtypes

# U.S. County Geographic Centers

In [ ]:
# 4. County Centers (by geographic center latitude and longitude)
countyCenterDataReadMeURL = "https://github.com/btskinner/spatial/blob/master/data/county_centers.csv"

# This is where the county center data .CSV lives locally...:
countyCenterDataFilepath = "Resources/county_centers.csv"

print(f"{countyCenterDataFilepath} is {round(os.path.getsize(countyCenterDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{countyCenterDataReadMeURL}")

In [ ]:
# Read CSV into countyCenterData DataFrame
countyCenterData = pd.read_csv(countyCenterDataFilepath)
countyCenterData

In [ ]:
# Merge countyCenterData with reorganized_final_merged_caseData to create heatmap DataFrame
heatmapData = pd.merge(reorganized_final_merged_caseData, countyCenterData, how = "left", left_on = "FIPS", right_on = "fips", on = None, sort = False, copy = True, indicator = False, validate = None)
heatmapData

In [ ]:
# Drop duplicate and/or irrelevant columns
heatmapData.drop(columns = ["fips", "clon00", "clat00", "pclon00", "pclat00", "pclon10", "pclat10"], inplace = True)
heatmapData.head()

In [ ]:
# Rename columns
heatmapData = heatmapData.rename(columns = {"clon10":"Longitude", "clat10":"Latitude"})
heatmapData.head()

In [ ]:
# Rearrange columns
heatmapData = heatmapData[["Date", "FIPS", "County", "State", "SQMI", "PopEst", "PopDens",
                           "Cases", "CaseRate", "Deaths", "DeathRate", "Latitude", "Longitude"]]
heatmapData.head()

In [ ]:
# Due diligence to check for missing data
heatmapData.count()

In [ ]:
# Manually add missing latitude and longitude coordinates:

# Kusilvak Census Area (Alaska)
heatmapData["Latitude"][81] = 62.09
heatmapData["Longitude"][81] = -163.53

# Oglala Lakota Census Area (South Dakota)
heatmapData["Latitude"][2404] = 43.33
heatmapData["Longitude"][2404] = -102.55

# New York City Aggregate (New York)
heatmapData["Latitude"][3131] = 40.7420
heatmapData["Longitude"][3131] = -73.9073

heatmapData.tail()

# U.S. COVID-19 Cases per 100,000 People on July 14, 2020

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key = g_key)

In [ ]:
# Create heatmap for CaseRate

# Store latitude and longitude in locations
locations = heatmapData[["Latitude", "Longitude"]]

# Convert case rates to float
caseRate = heatmapData["CaseRate"].astype(float)

# Plot Heatmap (U.S. geographic center is 39.8333, -98.5855)
fig = gmaps.figure(zoom_level = 4.1, center = (37.8, -98.6))

# Set max intensity to highest case rate found in the dataset
max_intensity = heatmapData["CaseRate"].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = caseRate, 
                                 dissipating = False, max_intensity = max_intensity,
                                 point_radius = 1.2, gradient = ['white', 'red'])



# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# U.S. COVID-19 Deaths per 100,000 People on July 14, 2020

In [ ]:
# Create heatmap for DeathRate

# Store latitude and longitude in locations
locations = heatmapData[["Latitude", "Longitude"]]

# Convert death rates to float
deathRate = heatmapData["DeathRate"].astype(float)

# Plot Heatmap (U.S. geographic center is 39.8333, -98.5855)
fig = gmaps.figure(zoom_level = 4.1, center = (37.8, -98.6))

# Set max intensity to mean death rate found in the dataset
max_intensity = heatmapData["DeathRate"].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = deathRate, 
                                 dissipating = False, max_intensity = max_intensity,
                                 point_radius = 0.8, gradient = ['white', 'red'])

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
# Matt's code ends here

In [ ]:
# Emerson's code starts here

# U.S. Mask-Wearing Survey

In [ ]:
# 5. mask-wearing survey
maskWearingDataReadMeURL = "https://github.com/nytimes/covid-19-data/blob/master/README.md"
# This is where the census data .CSV lives locally...:
maskWearingDataFilepath = "Resources/mask-use-by-county-exp.csv"

print(f"{maskWearingDataFilepath} is {round(os.path.getsize(maskWearingDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{maskWearingDataReadMeURL}")

In [ ]:
maskWearingData = pd.read_csv(maskWearingDataFilepath)
maskWearingData

In [ ]:
maskWearingData.dtypes

In [ ]:
scale = 10
divisions = 5
interval = scale / (divisions-1)
print(f"This will use the results of the NYT survey to score each county on a scale from 0 to {scale} where")
print(f"never = 0")
print(f"rarely = {interval}")
print(f"sometimes = {interval*2}")
print(f"frequently = {interval*3}")
print(f"always = {interval*4}")

In [ ]:
# maskWearingScore = rarely*interval + sometimes*interval*2 + frequently*interval*3 + always*interval*4

In [ ]:
# read in mask use by county csv file
file_to_load = "Resources/mask-use-by-county-exp.csv"
mask_counties = pd.read_csv(file_to_load)
mask_counties

In [ ]:
# total number of counties 
total_counties = mask_counties["COUNTYFP"].nunique()
total_counties_df = pd.DataFrame([total_counties], columns = ["Total Counties"])
total_counties_df

In [ ]:
# Average Never- Mask 
# never_mask = mask_counties["NEVER"].apply(percentages_to_floats).mean()
# never_mask

In [ ]:
def percentages_to_floats(percentage):
    string = percentage[0:-1]
    return float(string) 

In [ ]:
# Average Never - Mask 
never_mask = mask_counties["NEVER"].apply(percentages_to_floats).mean()
never_mask

In [ ]:
# Average Rarely - Mask 
rarely_mask = mask_counties["RARELY"].apply(percentages_to_floats).mean()
rarely_mask

In [ ]:
# Average Sometimes - Mask 
sometimes_mask = mask_counties["SOMETIMES"].apply(percentages_to_floats).mean()
sometimes_mask 

In [ ]:
# Average Frequently - Mask 
frequently_mask = mask_counties["FREQUENTLY"].apply(percentages_to_floats).mean()
frequently_mask

In [ ]:
# Average Always - Mask 
always_mask = mask_counties["ALWAYS"].apply(percentages_to_floats).mean()
always_mask

In [ ]:
# Create DataFrame of mask wearing mean percentages
summary_of_mask_usage_df = pd.DataFrame({"NEVER": [never_mask], 
                                        "RARELY": [rarely_mask], 
                                        "SOMETIMES": [sometimes_mask], 
                                        "FREQUENTLY": [frequently_mask], 
                                        "ALWAYS": [always_mask]})


print(summary_of_mask_usage_df)
# summary_of_mask_usage_df.style.format("{:.1%}")
pd.options.display.float_format = '{:,.2f}%'.format

In [ ]:
# updated mask score to include never, rarely, sometimes, frequently & always based on a 0, 2.5, 5, 7.5, 10 scale 
mask_counties["Mask Score"] = mask_counties["NEVER"].apply(percentages_to_floats) * 0 + mask_counties["RARELY"].apply(percentages_to_floats) * 2.5 + mask_counties["SOMETIMES"].apply(percentages_to_floats) * 5.0 + mask_counties["FREQUENTLY"].apply(percentages_to_floats) *7.5 + mask_counties["ALWAYS"].apply(percentages_to_floats) *10 
mask_counties

In [ ]:
# Descriptive 
mask_counties["Mask Score"].describe()

In [ ]:
# read in us-counties-csv in order to tick & tie the fips to the state names, and then start connecting to the mask score
counties = pd.read_csv(caseDataFilepath).dropna()
counties["fips"] = counties["fips"].apply(int)
counties = counties[["state", "fips"]]
counties = counties.rename({
    "fips": "COUNTYFP"
},axis = 1)
counties

In [ ]:
# merge dataframes in order to get Mask Score to align with state 
merged_df = mask_counties.merge(counties)
merged_df 

In [ ]:
# mask score sorted highest to lowest
merged_df = merged_df.sort_values("Mask Score", ascending = False)
merged_df 

# is there a connection between a higher or lower mask score based on location? Perhaps the geo map can provide additional info. 

In [ ]:
x_values = merged_df["state"]
y_values = merged_df["Mask Score"]

In [ ]:
# plt.bar(x_values, y_values)
# plt.xticks(rotation=90)
# plt.show()

# U.S. Mask Wearing Scores by State on July 14, 2020

In [ ]:
# created bar chart to demonstrate visual of mask score 
fig, ax = plt.subplots(figsize = (20,10))
ax.barh(x_values, y_values, color = "royalblue")
plt.xticks(rotation = 45)
plt.xticks(fontsize = 10)
ax.set_title("Mask Score\nby state as of July 14, 2020", fontsize = 18)
plt.xlabel("Mask Score", fontsize = 14)
# plt.ylabel("State", fontsize = 14)
# ax.grid()
plt.savefig("MaskWearingScoresByState.png")
plt.xlim

# U.S. Mask Wearing Scores by County on July 14, 2020

In [ ]:
# Merge heatmapData and merged_df
maskHeatMap = pd.merge(heatmapData, merged_df, how = "left", left_on = "FIPS", right_on = "COUNTYFP", on = None,
                       sort = False, copy = True, indicator = False, validate = None)
maskHeatMap.head()

In [ ]:
# Create heatmap for Mask Score

# Store latitude and longitude in locations
locations = maskHeatMap[["Latitude", "Longitude"]]

# Convert mask scores to float
mask_score = maskHeatMap["Mask Score"].astype(float)

# Plot Heatmap (U.S. geographic center is 39.8333, -98.5855)
fig = gmaps.figure(zoom_level = 4.1, center = (37.8, -98.6))

# Set max intensity to max mask score found in the dataset
max_intensity = merged_df["Mask Score"].max() * 6

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = mask_score, max_intensity = max_intensity, dissipating = False,
                                 point_radius = 0.8, gradient = ['white', 'blue'])

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
# Emerson's code ends here

In [ ]:
# Aleena's code starts here

# Summary DataFrame

In [ ]:
# Create GBDf
columnNames = ["FIPS", "County", "State", "Pop", "PopDens", "MskScore", "CaseRate", "DeathRate"]
GBDf = pd.DataFrame(columns = columnNames)
# placeholderData = ["01001", "Autauga", "Alabama", "55869", "94.0", "7.51", "0", "1335", "32"]
placeholderData = {"FIPS":"01001", "County":"Autauga", "State":"Alabama", "Pop":55869, "PopDens":94.0, "MskScore":7.51, "CaseRate":1335, "DeathRate":32}
GBDf = GBDf.append(placeholderData, ignore_index = True)
GBDf

In [ ]:
# Convert caseData FIPS values from float to int
GBDf.FIPS = GBDf.FIPS.astype(np.int64)
GBDf.dtypes

In [ ]:
# Merge GBDf DataFrame with reorganized_final_merged_caseData Dataframe
GBDf = pd.merge(reorganized_final_merged_caseData, GBDf, how = "left", left_on = "FIPS", right_on = "FIPS", on = None, sort = False, copy = True, indicator = False, validate = None)
GBDf

In [ ]:
# Drop duplicate and/or irrelevant columns
GBDf.drop(columns = ["Date", "SQMI", "Cases", "Deaths", "County_y", "State_y", "Pop", "PopDens_y", "MskScore",
                     "CaseRate_y", "DeathRate_y"], inplace = True)
GBDf.head()

In [ ]:
# Rename columns
GBDf = GBDf.rename(columns = {"County_x":"County", "State_x":"State", "PopEst":"Pop", "PopDens_x":"PopDens",
                              "CaseRate_x":"CaseRate", "DeathRate_x":"DeathRate"})
GBDf.head()

In [ ]:
# Create variable to hold mask score from mask score DataFrame
mskScore = merged_df["Mask Score"] / 100

In [ ]:
# Add new column to hold mask score
GBDf["MskScore"] = mskScore
GBDf.head()

In [ ]:
# Rearrange columns
GBDf = GBDf[["FIPS", "County", "State", "Pop", "PopDens", "MskScore", "CaseRate", "DeathRate"]]
GBDf

In [ ]:
GBDf.dtypes

In [ ]:
# Create formatted/clean dataframe to hold values from GBDf
formatted_GBDf = GBDf[["FIPS","County","State","Pop","PopDens", "MskScore", "CaseRate", "DeathRate"]].copy()
formatted_GBDf

In [ ]:
formatted_GBDf = formatted_GBDf.sort_values("FIPS", ascending = True)
formatted_GBDf

In [ ]:
# Reset index in place
formatted_GBDf.reset_index(drop = True, inplace = True)
formatted_GBDf

In [ ]:
# Convert GBDf values to strings for cleaner formatted display
formatted_GBDf["Pop"] = formatted_GBDf["Pop"].map("{:,}".format)
formatted_GBDf["PopDens"] = formatted_GBDf["PopDens"].map("{:,.2f}".format)
formatted_GBDf["MskScore"] = formatted_GBDf["MskScore"].map("{:,.2f}".format)
formatted_GBDf["CaseRate"] = formatted_GBDf["CaseRate"].map("{:,.2f}".format)
formatted_GBDf["DeathRate"] = formatted_GBDf["DeathRate"].map("{:,.2f}".format)
formatted_GBDf

In [ ]:
# Export to CSV
formatted_GBDf.to_csv("formatted_GBDf.csv", index = False, header = True)

# U.S. Mask Score vs. Population Density (with linear regression)

In [ ]:
# Do areas of higher population density have higher mask scores?

# Retrieve mask score and population density data
mskScore = GBDf["MskScore"]
popDens = GBDf["PopDens"]
n = len(GBDf)

# Perform a linear regression on population density versus mask scores
slope, int, r, p, std_err = st.linregress(popDens, mskScore)

# Create equation of line to calculate predicted mask scores
fit = slope * popDens + int

# Create equation in string formats to print on scatter plot
equation = "y = " + str(round(slope, 2)) + "x + " + str(round(int, 2))

# Define scatter plot size
plt.figure(figsize = (7, 7))

# Plot x and y values on scatter plot
plt.scatter(popDens, mskScore, marker=".", color="black")

# Plot linear regression line on scatter plot
plt.plot(popDens, fit, "--", color = "red")

# Define linear regression line and print on scatter plot
plt.annotate(equation, (2500, 7), fontsize = 14, color = "red")

# Define scatter plot title date, and x and y labels (and their font sizes)
# medPopDens = GBDf["PopDens"].median()

plt.title(f"Mask Scores vs. Population Density\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.xlabel("Population Density", fontsize = 18)
plt.ylabel("Mask Score", fontsize = 18)
plt.xlim(0, 28000)
plt.ylim(3,10)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

print(f"The r-value is: {r}")
plt.savefig("MaskWearingVsPopDensity.png")
plt.show()

In [ ]:
# Same as above but zoomed in
# Do areas of higher population density have higher mask scores?

# Retrieve mask score and population density data
mskScore = GBDf["MskScore"]
popDens = GBDf["PopDens"]
n = len(GBDf)

# Perform a linear regression on population density versus mask scores
slope, int, r, p, std_err = st.linregress(popDens, mskScore)

# Create equation of line to calculate predicted mask scores
fit = slope * popDens + int

# Create equation in string formats to print on scatter plot
equation = "y = " + str(round(slope, 4)) + "x + " + str(round(int, 4))

# Define scatter plot size
plt.figure(figsize = (7, 7))

# Plot x and y values on scatter plot
plt.scatter(popDens, mskScore, marker=".", color="black")

# Plot linear regression line on scatter plot
plt.plot(popDens, fit, "--", color = "red")

# Define linear regression line and print on scatter plot
plt.annotate(equation, (1750, 6.5), fontsize = 14, color = "red")

# Define scatter plot title date, and x and y labels (and their font sizes)
medPopDens = GBDf["PopDens"].median()

plt.title(f"Mask Scores vs. Population Density\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.xlabel("Population Density", fontsize = 18)
plt.ylabel("Mask Score", fontsize = 18)
plt.xlim(0, 3000)
plt.ylim(3,10)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

print(f"The r-value is: {r}")
plt.savefig("ZOOMEDMaskWearingVsPopDensity.png")
plt.show()

In [ ]:
# Aleena's code ends here

In [ ]:
# Paul's code starts here

# Cumulative COVID case rate per 100,000 population vs. Population density

In [ ]:
# Create a pair of scatterplots for case rate vs population density
# The proper form for a graph title is "y-axis variable vs. x-axis variable," so:
# case rate vs population density and
# death rate vs population density
# independent variable goes on the x-axis: population density
#   dependent variable goes on the y-axis: COVID case and death rates

# Set x values to be used in both scatterplots
n = len(GBDf)
xValues = GBDf.loc[:, "PopDens"]
CyValues = GBDf.loc[:, "CaseRate"]
DyValues = GBDf.loc[:, "DeathRate"]
print(f"max. pop. dens. (x): {max(xValues)}")
print(f"max. cases (Cy): {max(CyValues)}")
print(f"max. deaths (Dy): {max(DyValues)}")

# Set width of x axis and height of y axis in both graphs
xLimMax = 28000
CyLimMax = 20000
DyLimMax = 400

# Define scatter plot size
plt.figure(figsize = (7, 7))

# plot cases (C)
plt.title(f"COVID-19 Cases vs. Population Density\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.scatter(xValues, CyValues, marker = ".", color = "black")
plt.xlim(0, xLimMax)
plt.ylim(0, CyLimMax)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

# now do linear regression
(Cslope, Cintercept, Crvalue, Cpvalue, Cstderr) = linregress(xValues, CyValues)
Cline_eq = "y = " + str(round(Cslope,2)) + "x + " + str(round(Cintercept,2))
plt.xlabel(f"Population Density (people per square mile)", fontsize = 18)
plt.ylabel(f"Cumulative Cases per 100,000 pop.", fontsize = 18)
Cregress_values = np.asarray(Cslope) * xValues + Cintercept
plt.plot(xValues,Cregress_values,"r-")

# Define linear regression line and print on scatter plot
plt.annotate(Cline_eq, (15000, 5000), fontsize = 14, color = "red")

plt.savefig("PopDensVsCOVIDCaseRate.png")
print(f"The r-value is: {Crvalue}")
print(f"case slope: {Cslope}")
plt.show()

In [ ]:
# Same as above but zoomed in
# Create a pair of scatterplots for case rate vs population density
# The proper form for a graph title is "y-axis variable vs. x-axis variable," so:
# case rate vs population density and
# death rate vs population density
# independent variable goes on the x-axis: population density
#   dependent variable goes on the y-axis: COVID case and death rates

# Set x values to be used in both scatterplots
n = len(GBDf)
xValues = GBDf.loc[:, "PopDens"]
CyValues = GBDf.loc[:, "CaseRate"]
DyValues = GBDf.loc[:, "DeathRate"]
print(f"max. pop. dens. (x): {max(xValues)}")
print(f"max. cases (Cy): {max(CyValues)}")
print(f"max. deaths (Dy): {max(DyValues)}")

# Set width of x axis and height of y axis in both graphs
xLimMax = 5000
CyLimMax = 5000
DyLimMax = 400

# Define scatter plot size
plt.figure(figsize = (7, 7))

# plot cases (C)
plt.title(f"COVID-19 Cases vs. Population Density\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.scatter(xValues, CyValues, marker = ".", color = "black")
plt.xlim(0, xLimMax)
plt.ylim(0, CyLimMax)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

# now do linear regression
(Cslope, Cintercept, Crvalue, Cpvalue, Cstderr) = linregress(xValues, CyValues)
Cline_eq = "y = " + str(round(Cslope,2)) + "x + " + str(round(Cintercept,2))
plt.xlabel(f"Population Density (people per square mile)", fontsize = 18)
plt.ylabel(f"Cumulative Cases per 100,000 pop.", fontsize = 18)
Cregress_values = np.asarray(Cslope) * xValues + Cintercept
plt.plot(xValues,Cregress_values,"r-")

# Define linear regression line and print on scatter plot
plt.annotate(Cline_eq, (3000, 2000), fontsize = 14, color = "red")

plt.savefig("ZOOMEDPopDensVsCOVIDCaseRate.png")
print(f"The r-value is: {Crvalue}")
print(f"case slope: {Cslope}")
plt.show()

# Cumulative COVID death rate per 100,000 population vs. Population density

In [ ]:
# Create a pair of scatterplots for case rate vs population density
# The proper form for a graph title is "y-axis variable vs. x-axis variable," so:
# case rate vs population density and
# death rate vs population density
# independent variable goes on the x-axis: population density
#   dependent variable goes on the y-axis: COVID case and death rates

# Set x values to be used in both scatterplots
n = len(GBDf)
xValues = GBDf.loc[:, "PopDens"]
CyValues = GBDf.loc[:, "CaseRate"]
DyValues = GBDf.loc[:, "DeathRate"]
print(f"max. pop. dens. (x): {max(xValues)}")
print(f"max. cases (Cy): {max(CyValues)}")
print(f"max. deaths (Dy): {max(DyValues)}")

# Set width of x axis and height of y axis in both graphs
xLimMax = 28000
CyLimMax = 5000
DyLimMax = 400

# Define scatter plot size
plt.figure(figsize = (7, 7))

# plot cases (C)
plt.title(f"COVID-19 Deaths vs. Population Density\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.scatter(xValues, DyValues, marker = ".", color = "black")
plt.xlim(0, xLimMax)
plt.ylim(0, DyLimMax)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

# now do linear regression
(Dslope, Dintercept, Drvalue, Dpvalue, Dstderr) = linregress(xValues, DyValues)
Dline_eq = "y = " + str(round(Dslope,2)) + "x + " + str(round(Dintercept,2))
plt.xlabel(f"Population Density (people per square mile)", fontsize = 18)
plt.ylabel(f"Cumulative Deaths per 100,000 pop.", fontsize = 18)
Dregress_values = np.asarray(Dslope) * xValues + Dintercept
plt.plot(xValues,Dregress_values,"r-")

# Define linear regression line and print on scatter plot
plt.annotate(Dline_eq, (15000, 100), fontsize = 14, color = "red")

plt.savefig("PopDensVsCOVIDDeathRate.png")
print(f"The r-value is: {Drvalue}")
print(f"case slope: {Dslope}")
plt.show()

In [ ]:
# Create a scatterplots for death rate vs case rate
# The proper form for a graph title is "y-axis variable vs. x-axis variable," so:
# death rate vs case rate
# independent variable goes on the x-axis: case rate
#   dependent variable goes on the y-axis: death rate

# Set x values to be used in both scatterplots
n = len(GBDf)
xValues = GBDf.loc[:, "CaseRate"]
yValues = GBDf.loc[:, "DeathRate"]
print(f"max. cases (x): {max(xValues)}")
print(f"max. deaths (y): {max(yValues)}")

# Set axis limits in both graphs (shared y)
xLimMax = 15000
yLimMax = 400

# Cumulative COVID case rate per 100,000 population vs. Mask-wearing Score

In [ ]:
# Create a scatterplot for case rate vs mask-wearing score
# The proper form for a graph title is "y-axis variable vs. x-axis variable," so:
# case rate vs population density and
# death rate vs population density
# independent variable goes on the x-axis: mask-wearing score
#   dependent variable goes on the y-axis: COVID case and death rates

# Set x and y values
n = len(GBDf)
xValues = GBDf.loc[:, "MskScore"]
CyValues = GBDf.loc[:, "CaseRate"]
DyValues = GBDf.loc[:, "DeathRate"]
print(f"max. mask score (x): {max(xValues)}")
print(f"max. cases (Cy): {max(CyValues)}")
print(f"max. deaths (Dy): {max(DyValues)}")

# Set width of x axis and height of y axis in both graphs
xLimMax = 10
CyLimMax = 20000
DyLimMax = 400

# Define scatter plot size
plt.figure(figsize = (7, 7))

# plot cases (C)
plt.title(f"COVID-19 Cases vs. Mask Wearing Score\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.scatter(xValues, CyValues, marker = ".", color = "black")
plt.xlim(2, xLimMax)
plt.ylim(0, CyLimMax)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

# now do linear regression
(Cslope, Cintercept, Crvalue, Cpvalue, Cstderr) = linregress(xValues, CyValues)
Cline_eq = "y = " + str(round(Cslope,2)) + "x + " + str(round(Cintercept,2))
plt.xlabel(f"Mask Wearing Score", fontsize = 18)
plt.ylabel(f"Cumulative Cases per 100,000 pop.", fontsize = 18)
Cregress_values = np.asarray(Cslope) * xValues + Cintercept
plt.plot(xValues,Cregress_values,"r-")

# Define linear regression line and print on scatter plot
plt.annotate(Cline_eq, (3, 5000), fontsize = 14, color = "red")

plt.savefig("MaskWearingVsCOVIDCaseRate.png")
print(f"The r-value is: {Crvalue}")
print(f"case slope: {Cslope}")
plt.show()

# Cumulative COVID death rate per 100,000 population vs. Population density

In [ ]:
# Create a scatterplot for death rate vs mask-wearing score
# The proper form for a graph title is "y-axis variable vs. x-axis variable," so:
# case rate vs population density and
# death rate vs population density
# independent variable goes on the x-axis: mask-wearing score
#   dependent variable goes on the y-axis: COVID death rates

# Set x and y values
xValues = GBDf.loc[:, "MskScore"]
yValues = GBDf.loc[:, "DeathRate"]
print(f"max. mask score (1) (x): {max(xValues)}")
print(f"max. deaths (Dy): {max(yValues)}")

# Set width of x axis and height of y axis in both graphs
xLimMax = 10
CyLimMax = 15000
DyLimMax = 400

# Define scatter plot size
plt.figure(figsize = (7, 7))

# plot deaths (D)
plt.title(f"COVID-19 Deaths vs. Mask Wearing Score\nby county as of July 14, 2020 (n={n})", fontsize = 20)
plt.scatter(xValues, yValues, marker = ".", color = "black")
plt.xlim(2, xLimMax)
plt.ylim(0, yLimMax)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

# now do linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(xValues, yValues)
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.xlabel(f"Mask Wearing Score", fontsize = 18)
plt.ylabel(f"Cumulative Deaths per 100,000 pop.", fontsize = 18)
regress_values = np.asarray(slope) * xValues + intercept
plt.plot(xValues,regress_values,"r-")

# Define linear regression line and print on scatter plot
plt.annotate(line_eq, (3, 100), fontsize = 14, color = "red")

plt.savefig("MaskWearingVsCOVIDDeathRate.png")
print(f"The r-value is: {rvalue}")
print(f"death slope: {slope}")
plt.show()

In [ ]:
# Paul's code ends here